Helper functions from 
https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb

In [1]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = 'Input/input{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("http://adventofcode.com/2017/day/{}/input".format(day))

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

assert tuple(transpose(((1, 2, 3), (4, 5, 6)))) == ((1, 4), (2, 5), (3, 6))
assert first('abc') == first(['a', 'b', 'c']) == 'a'
assert cat(['a', 'b', 'c']) == 'abc'
assert (groupby(['test', 'one', 'two', 'three', 'four'], key=len) 
        == {3: ['one', 'two'], 4: ['test', 'four'], 5: ['three']})

#data = Input(1).read()[:-1]

day1

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

In [ ]:
# day 1
data = Input(1).read()[:-1] # get rid of the newline character
def captcha(string,j):
    # j indicates part 1 or two
    l = len(string)
    res = 0
    if j == 1:
        diff = 1
    if j == 2:
        diff = l//2
    for i in range(l):
        if string[i] == string[(i+diff)%l]:
            res += int(string[i])        
    return res
(captcha(data,1), captcha(data,2))       

day 2

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

In [ ]:
lines = Input(2).readlines()
def checksum1(lines):
    checksum = 0
    for line in lines:
        nums = line.split()
        temp = [int(num) for num in nums]
        checksum += max(temp) - min(temp)
    return checksum
checksum1(data)

In [ ]:
def checkline(nums):
    l = len(nums)
    for i in range(l):
        for j in range(i+1,l):
            if (nums[i]%nums[j]==0 or nums[j]%nums[i]==0):
                return max(nums[i]//nums[j], nums[j]//nums[i])    

In [ ]:
def checksum2(lines):
    checksum = 0
    for line in lines:
        nums = line.split()
        temp = [int(num) for num in nums]
        checksum += checkline(temp)
    return checksum
checksum2(lines)

Day 3

In [ ]:
N = 265149
width = math.ceil(N**(1/2))
horizontal = 514/2-(width**2 - N)
vertical = 514/2
dist = horizontal + vertical
dist

In [ ]:
def around(nums, i, j):
    nums[i][j] = nums[i-1][j-1] + nums[i-1][j] + nums[i-1][j+1] +\
        nums[i][j-1] + nums[i][j+1] + \
        nums[i+1][j-1] + nums[i+1][j] + nums[i+1][j+1]

In [ ]:
l = 5;
nums = [[0]*l,[0,5,4,2,0],[0,10,1,1,0],[0,11,23,25,0],[0]*l]
for l in range(l,10,2):
    for i in range(l):
        nums[i].append(0)
        nums[i].insert(0,0)
    nums.insert(0,[0]*(l+2))
    nums.append([0]*(l+2))
    i = l-1
    j = l
    for i in range(l-1,0,-1):
        around(nums,i,j)
    for j in range(l-1,0,-1):
        around(nums,i,j)
    for i in range(2, l+1):
        around(nums,i,j)
    for j in range(2, l+1):
        around(nums,i,j)
print(l)
for num in nums:
    print(num)

Day 4

In [ ]:
lines = Input(4).readlines()
def checkAna(words):
    l = len(words)
    for i in range(l):
        word_i = sorted(words[i])
        for j in range(i+1,l):
            word_j = sorted(words[j])
            if word_i == word_j:                
                return False
    return True      

In [ ]:
count = 0
for line in lines:
    words = line.split()
    unique = set(words)
    if len(unique) == len(words) and checkAna(words):
        count += 1    

In [ ]:
count

Day 5
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

In [ ]:
lines = Input(5).readlines()

for i in range(len(lines)):
    lines[i] = int(lines[i])

#lines = [0,3,0,1,-3]
ind = 0
count = 0
while 0<= ind < len(lines):
    jump = lines[ind]
    if jump >= 3:
        lines[ind] -= 1
    else:
        lines[ind] += 1
    ind += jump    
    count += 1
count

Day 6
The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In [ ]:
lines = Input(6).readlines()
nums = lines[0].split()
for i in range(len(nums)):
    nums[i] = int(nums[i])

In [ ]:
cycles = []
l = len(nums)
count = 0
while nums not in cycles:
    count += 1
    cycles.append(nums[:])
    m = max(nums)
    ind = nums.index(m)    
    nums[ind] = 0
    for i in range(m):
        ind += 1
        nums[ind%l] += 1

count 

In [ ]:
count - cycles.index(nums)

Day 7

In [ ]:
lines = Input('7_1').readlines()

In [ ]:
tops = set()
bottom = set()
for line in lines:
    words = line.split()
    bottom.add(words[0])
    if len(words) > 2:
        for i in range(3,len(words)):
            tops.add(words[i].replace(",", ""))


In [ ]:
tops.symmetric_difference(bottom)

In [ ]:
weight = {}
for line in lines:
    words = line.split()
    weight[words[0]] = int(words[1].replace("(","").replace(")",""))   

In [ ]:
weight

In [ ]:
struct = {}
for line in lines:
    words = line.split()
    if len(words) >2:
        struct[words[0]] = [words[i].replace(",","") for i in range(3,len(words))]
struct

In [ ]:
class TreeNode():
    def __init__(self,initWeight=0):
        self.children = list()
        self.weight = initWeight

    def addChild(self, weight):
        self.children.append(TreeNode(weight))

In [ ]:
root = TreeNode(weight['tknk'])
prevs = struct['tknk']
[root.addChild(weight[word]) for word in prevs]
temp = root
while prevs:
    
    for word in prevs:
        root.addChild(weight[word])


In [ ]:
root.children[0]

In [ ]:
root.children[0].weight

Day 8

In [ ]:
# read input
lines = Input(8).readlines()
words = lines[0].split()
words

In [ ]:
# create a dictionary with each variable following the instructions
# return the maximum value for part 1
# and store the maximum value encountered for part 2
nums = {}
m = 0
for line in lines:
    words = line.split()
    cond = words[4]
    num = words[0]
    if cond not in nums:
        nums[cond] = 0
    if num not in nums:
        nums[num] = 0
    if eval(str(nums[cond])+words[5]+words[6]):
        flag = -1
        if words[1] == 'inc':
            flag = 1
        nums[num] += flag*int(words[2])
    m = max(m, max(nums.values()))
m1 = max(nums.values())

In [ ]:
m

Day 9

In [ ]:
# read input
lines = Input(9).readlines()
s = lines[0]

In [ ]:
# remove ! and its next character
i = 0
l = len(s)
while i < l:
    c = s[i]
    if c == '!':
        s = s[:i] + s[i+2:]
        i -= 1
    l = len(s)
    i += 1

In [ ]:
# remove trash with <>, and count how many chars removed
i = 0
l = len(s)
st = False
count = 0
while i < l:
    c = s[i]
    if c == '<' and (not st):
        st = True
        si = i
    if c == '>' and st:
        st = False
        s = s[:si] + s[i+1:]        
        count += i-si-1
        i = si
    else:
        i += 1
    l = len(s)
count   

In [ ]:
# count total score for groups with {}, each level up has 1 more score
i = 0
l = len(s)
num = 0
res = 0
while i < l:
    c = s[i]
    if c == '{':
        num += 1
        res += num
    if c == '}':
        num -= 1
    l = len(s)
    i += 1
res

Day 10

In [ ]:
l = 256
list = []
for i in range(l):
    list.append(i)
length = [63,144,180,149,1,255,167,84,125,65,188,0,2,254,229,24]

In [ ]:
s = 0
cur = 0
for le in length:
    start = cur
    end = cur + le - 1
    if end < l:
        rev = list[start:end+1]
        list[start:end+1] = rev[::-1]
    else: # when we have wrap
        end = end%l
        rev = list[start:] + list[:end+1]
        rev = rev[::-1]
        list[start:] = rev[:l-start]
        list[:end+1] = rev[l-start:]
    cur += le + s
    cur %= l
    s+= 1

In [ ]:
list[0]*list[1]

In [ ]:
l = 256
list = []
for i in range(l):
    list.append(i)
chars = '63,144,180,149,1,255,167,84,125,65,188,0,2,254,229,24'
length = [17, 31, 73, 47, 23]
s = 0
cur = 0
for k in range(64):
    for i in range(len(length)+len(chars)):
        if i < len(chars):
            le = ord(chars[i])
        else:
            le = length[i-len(chars)]   
        start = cur
        end = cur + le - 1
        if end < l:
            rev = list[start:end+1]
            list[start:end+1] = rev[::-1]
        else:
            end = end%l
            rev = list[start:] + list[:end+1]
            rev = rev[::-1]
            list[start:] = rev[:l-start]
            list[:end+1] = rev[l-start:]
        cur += le + s
        cur %= l
        s+= 1

In [ ]:
ind = 0
ans = ''
for i in range(16):
    ind = 16*i
    resi = 0
    for j in range(16):
        resi ^= list[ind]
        ind += 1
    ansi = hex(resi).split('x')[-1]
    if len(ansi) < 2:
        ansi = '0' + ansi
    ans += ansi

In [ ]:
ans

Day 11

In [ ]:
lines = Input(11).readlines()
steps = lines[0].split(',')
steps[-1] = steps[-1].replace('\n','')
steps[-1]

part I

In [ ]:
count = [steps.count(d) for d in ['n','ne','se','s','sw','nw']]
[count[j]-count[j+3] for j in range(3)]

In [ ]:
403+405

part II

In [ ]:
m = 0
for i in range(len(steps)):
    count = [steps[:i].count(d) for d in ['n','ne','se','s','sw','nw']]
    diff = [count[j]-count[j+3] for j in range(3)]
    num = abs(diff[0]+diff[1])
    m = max(m,num)
m   

In [69]:
lines = Input('12').readlines()

In [70]:
dict = {}
for line in lines:
    line = line.split()
    dict[line[0]] = line[2:]
for value in dict.values():
    for i in range(len(value)):
        value[i] = value[i].replace(',','')

In [71]:
len(dict)

2000

In [72]:
s = set()
cur = set('0')
while len(cur) != 0:
    temp = cur
    cur = set()
    for key in temp:
        if key not in s:
            cur.update(dict[key])
    s.update(temp)
len(s)

175

In [73]:
len(dict)
count = 0
#while len(dict) != 0:
s = set()
cur = set(list(dict.keys())[0])
while len(cur) != 0:
    temp = cur
    cur = set()
    for key in temp:
        if key not in s:
            cur.update(dict[key])
    s.update(temp)
for key in s:
    del dict[key]
count += 1
len(dict)

1659

In [75]:
'0' in s

True

In [67]:
cur = set(list(dict.keys())[0])
cur

{'0', '1', '6', '9'}

In [68]:
dict['0']

KeyError: '0'

In [10]:
for key in temp:
    cur.update(dict[key])

In [11]:
cur

{'1774', '199'}

In [12]:
s.update(cur)

In [13]:
s

{'1774', '199'}

In [14]:
cur

{'1774', '199'}

In [15]:
cur = set()

In [18]:
assert len(cur) == 0

Day 13

In [118]:
lines = Input(13).readlines()
list = []
for line in lines:
    list.append(line.replace(':','').split())

In [119]:
def pos(l,d):
    pos = l%(2*d-2)
    return pos

In [120]:
ans = 0
for [l,d] in list:
    if pos(int(l),int(d)) == 0:
        print(l,d)
        ans+= int(l)*int(d)
ans

0 5
28 8
38 20
44 12
78 14


2604

In [121]:
def caught(delay, list):
    for [l,d] in list:
        if pos(int(l)+delay,int(d)) == 0:
            return True
    return False

In [116]:
delay = 1
while caught(delay, list):
    delay += 1

In [117]:
delay

3941460